# 스탯티즈 Playlog 데이터를 크롤링하는 과정 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

hts_2018 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2018.csv', encoding='euc-kr')
hts_2019 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2019.csv', encoding='euc-kr')
hts_2020 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2020.csv', encoding='euc-kr')
hts_2021 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2021.csv', encoding='euc-kr')
hts = pd.concat([hts_2018,hts_2019,hts_2020,hts_2021], ignore_index=True)

player_2018 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2018.csv', encoding='euc-kr')
player_2019 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2019.csv', encoding='euc-kr')
player_2020 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2020.csv', encoding='euc-kr')
player_2021 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2021.csv', encoding='euc-kr')
player = pd.concat([player_2018,player_2019,player_2020,player_2021], ignore_index=True)

hitter_base_2018 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2018.csv', encoding='euc-kr')
hitter_base_2019 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2019.csv', encoding='euc-kr')
hitter_base_2020 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2020.csv', encoding='euc-kr')
hitter_base_2021 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2021.csv', encoding='euc-kr')
hitter_base = pd.concat([hitter_base_2018,hitter_base_2019,hitter_base_2020,hitter_base_2021], ignore_index=True)

team = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_팀.csv', encoding='euc-kr')
schedule_2021 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_경기일정_2021.csv', encoding='euc-kr')

In [ ]:
# 타자 기록이 있는 선수들과 등록 선수를 pcode와 연도를 기준으로 merge
hitter = hitter_base.merge(player, on=['PCODE','GYEAR'])

In [ ]:
# 이름을 개명한 선수에 대해서 같은 이름이 쓰이도록 처리
hitter.loc[hitter['NAME']=='한동민','NAME'] = '한유섬'
hitter.loc[hitter['NAME']=='백민기','NAME'] = '백동훈'
hitter.loc[hitter['NAME']=='윤대영','NAME'] = '윤형준'
hitter.loc[hitter['NAME']=='이병휘','NAME'] = '이유찬'
hitter.loc[hitter['NAME']=='나종덕','NAME'] = '나균안'
hitter.loc[hitter['NAME']=='김동욱','NAME'] = '김동명'
hitter.loc[hitter['NAME']=='이찬건','NAME'] = '이석훈'
hitter.loc[hitter['NAME']=='김지열','NAME'] = '김사연'

hitter['BIRTH']=0
# 제공받는 hitter와 스탯티즈의 연도별 선수 기록이 달라서 동명이인을 처리 못하는 사람들에 대해 미리 생년월일을 입력
# 기록이 다른 선수 명단 : 천성호, 이재원, 이주형, 김주현
hitter.loc[hitter['PCODE']==50054,'BIRTH'] = '1997-10-30'
hitter.loc[hitter['PCODE']==68106,'BIRTH'] = '1999-07-17'
hitter.loc[hitter['PCODE']==50167,'BIRTH'] = '2001-04-02'
hitter.loc[hitter['PCODE']==66749,'BIRTH'] = '1993-12-21'

In [ ]:
###############
# BeautifulSoup

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

# 데이터를 모을 dataframe 생성
data = pd.DataFrame(columns=['GYEAR','PCODE','NAME','T_ID','POSITION','day', 'vs', 'inn', 'pitcher', 'batter', 'P', 'result', 'before', 'after', 'LEV', 'REa', 'WPs','WPe', 'WPa'])
same_name_fail_list = []

for hitter_index in tqdm(range(len(hitter))):
  year = hitter.loc[hitter_index,'GYEAR']
  player_pcode = hitter.loc[hitter_index,'PCODE']
  player_name = hitter.loc[hitter_index,'NAME']
  player_team = hitter.loc[hitter_index,'T_ID']
  player_position = hitter.loc[hitter_index,'POSITION']

  # 선수와 연도에 맞는 플레이 로그 페이지
  wp = requests.get('http://www.statiz.co.kr/player.php?opt=6&sopt=0&name=' + player_name + '&re=0&da=1&year=' + str(year))
  soup = BeautifulSoup(wp.text, "html.parser")

  # 동명이인 문제로 페이지에 데이터가 없다면
  row_data = soup.find_all(class_='table table-striped table-responsive table-condensed table-bordered')
  if len(row_data)==0:
    print("\n동명이인 존재 : " + player_name)
    if hitter.loc[hitter_index,'BIRTH']!=0:
      print("생일로 찾음")
      wp = requests.get('http://www.statiz.co.kr/player.php?opt=6&sopt=0&name=' + player_name + '&birth=' + hitter.loc[hitter_index,'BIRTH'] + '&re=0&da=1&year=' + str(year))
      soup = BeautifulSoup(wp.text, "html.parser")
    else:  
      same_name_href_list=[]
      wp = requests.get('http://www.statiz.co.kr/player.php?name=' + player_name)
      soup = BeautifulSoup(wp.text, "html.parser")
      for same_name_list in soup.find(class_='table table-striped').find_all('a'):
        same_name_href_list.append(same_name_list.attrs['href'])
      flag=0
      for same_name_page in same_name_href_list:
        wp = requests.get('http://www.statiz.co.kr/'+same_name_page+'&opt=1&re=0')
        soup = BeautifulSoup(wp.text, "html.parser")
        # 현재 찾고 있는 선수의 기록과 현재 페이지의 선수 기록이 일치하면 같은 선수라는 의미이다. 
        d = str(hitter.loc[hitter_index,'GAMENUM'].item()) + str(hitter.loc[hitter_index,'PA'].item()) + str(hitter.loc[hitter_index,'AB'].item())
        if d in soup.text:
          flag=1
          # 동명이인의 선수에 대해서 생일을 기록해둔다.
          print(player_name+" 생일 저장")
          player_birth = same_name_page[-10:]
          hitter.loc[hitter['PCODE']==player_pcode, 'BIRTH'] = player_birth
          break
      # 찾은 선수의 페이지로 접근한다.
      if flag==1:
        print("기록과 일치하는 선수 찾음")
      else:
        print("기록과 일치하는 선수 못찾음")
        same_name_fail_list.append((year,player_pcode,player_name,player_team))
        continue
      wp = requests.get('http://www.statiz.co.kr/player.php?opt=6&'+same_name_page.split('?')[1]+'&re=0&da=1&year='+str(year))
      soup = BeautifulSoup(wp.text, "html.parser")

  # 데이터 테이블 접근
  row_data = soup.find_all(class_='table table-striped table-responsive table-condensed table-bordered')[0]
  row_data_tr = row_data.find_all('tr')[1:]

  # 행렬만들기
  rawdata_day = np.zeros(len(row_data_tr), dtype=object)
  rawdata_vs = np.zeros(len(row_data_tr), dtype=object)
  rawdata_inn = np.zeros(len(row_data_tr), dtype=object)
  rawdata_pitcher = np.zeros(len(row_data_tr),dtype=object)
  rawdata_batter = np.zeros(len(row_data_tr), dtype=object)
  rawdata_P = np.zeros(len(row_data_tr), dtype=object)
  rawdata_result = np.zeros(len(row_data_tr), dtype=object)
  rawdata_before = np.zeros(len(row_data_tr), dtype=object)
  rawdata_after = np.zeros(len(row_data_tr), dtype=object)
  rawdata_LEV = np.zeros(len(row_data_tr), dtype=object)
  rawdata_REa = np.zeros(len(row_data_tr), dtype=object)
  rawdata_WPs = np.zeros(len(row_data_tr), dtype=object)
  rawdata_WPe = np.zeros(len(row_data_tr), dtype=object)
  rawdata_WPa = np.zeros(len(row_data_tr), dtype=object)

  # 행 하나씩 접근, 헤더 생략
  for tr_index in range(len(row_data_tr)):
    rawdata_td_element = [text for text in row_data_tr[tr_index].stripped_strings]
    # P 칼럼의 값이 없을때도 있음.
    if len(rawdata_td_element) == 14:
      rawdata_td_element.insert(6,"")
    
    # 데이터 입력
    rawdata_day[tr_index] = rawdata_td_element[0]
    rawdata_vs[tr_index] = rawdata_td_element[1]
    rawdata_inn[tr_index] = rawdata_td_element[2]
    rawdata_pitcher[tr_index] = rawdata_td_element[3]
    rawdata_batter[tr_index] = rawdata_td_element[5]
    rawdata_P[tr_index] = rawdata_td_element[6]
    rawdata_result[tr_index] = rawdata_td_element[7]
    rawdata_before[tr_index] = rawdata_td_element[8]
    rawdata_after[tr_index] = rawdata_td_element[9]
    rawdata_LEV[tr_index] = rawdata_td_element[10]
    rawdata_REa[tr_index] = rawdata_td_element[11]
    rawdata_WPs[tr_index] = rawdata_td_element[12]
    rawdata_WPe[tr_index] = rawdata_td_element[13]
    rawdata_WPa[tr_index] = rawdata_td_element[14]

  tmp_data = pd.DataFrame()

  # tmp_data는 각 선수마다 데이터를 모은 후 data에 concat 하기 전 임시로 저장
  tmp_data['day'] = rawdata_day
  tmp_data['vs'] = rawdata_vs
  tmp_data['inn'] = rawdata_inn
  tmp_data['pitcher'] = rawdata_pitcher
  tmp_data['batter'] = rawdata_batter
  tmp_data['P'] = rawdata_P
  tmp_data['result'] = rawdata_result
  tmp_data['before'] = rawdata_before
  tmp_data['after'] = rawdata_after
  tmp_data['LEV'] = rawdata_LEV
  tmp_data['REa'] = rawdata_REa
  tmp_data['WPs'] = rawdata_WPs
  tmp_data['WPe'] = rawdata_WPe
  tmp_data['WPa'] = rawdata_WPa

  tmp_data['GYEAR'] = year
  tmp_data['PCODE'] = player_pcode
  tmp_data['NAME'] = player_name
  tmp_data['T_ID'] = player_team
  tmp_data['POSITION'] = player_position

  # 데이터 쌓기
  data = pd.concat([data,tmp_data], ignore_index=True)

  0%|          | 3/1098 [00:02<16:05,  1.13it/s]


동명이인 존재 : 이해창
이해창 생일 저장
기록과 일치하는 선수 찾음


  1%|          | 7/1098 [00:10<27:46,  1.53s/it]


동명이인 존재 : 이상호
이상호 생일 저장
기록과 일치하는 선수 찾음


  1%|          | 8/1098 [00:15<44:06,  2.43s/it]

...

 99%|█████████▉| 1090/1098 [28:44<00:12,  1.61s/it]


동명이인 존재 : 박건우
생일로 찾음


100%|█████████▉| 1095/1098 [28:52<00:04,  1.64s/it]


동명이인 존재 : 김상수
생일로 찾음


100%|██████████| 1098/1098 [28:58<00:00,  1.58s/it]


In [ ]:
data

,GYEAR,PCODE,NAME,T_ID,POSITION,day,vs,inn,pitcher,batter,P,result,before,after,LEV,REa,WPs,WPe,WPa
0,2018,60100,백창수,HH,외,2018-03-27,NC,8초,이민호,백창수,1-0,유격수 땅볼,무사 6:9,1사 6:9,0.88,-0.258,93.1,95.3,-0.022
1,2018,60100,백창수,HH,외,2018-03-28,NC,2초,구창모,백창수,2-3,볼넷,2사 2루 0:2,"2사 1,2루 0:2",1.10,0.122,71.9,70.7,0.012
2,2018,60100,백창수,HH,외,2018-03-28,NC,4초,구창모,백창수,0-0,3루수 땅볼,2사 1:2,이닝종료 1:2,0.51,-0.117,67.4,68.8,-0.013
3,2018,60100,백창수,HH,외,2018-03-28,NC,7초,유원상,백창수,2-3,유격수 땅볼 실책,[실책포함],무사 2:2,무사 1루 2:2,1.54,0.398,50.0,44.1
4,2018,60100,백창수,HH,외,2018-03-30,SK,2말,김태훈,백창수,0-1,백창수/10473 : 2루수 직선타 아웃,"1사 2,3루 1:1","2사 2,3루 1:1",1.57,-0.833,64.1,56.1,-0.080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205941,2021,79608,안치홍,LT,내,2021-08-25,KIA,2초,멩덴,안치홍,1-1,2루수 땅볼,1사 0:0,2사 0:0,0.65,-0.180,52.4,54.0,-0.016
205942,2021,79608,안치홍,LT,내,2021-08-25,KIA,4초,멩덴,안치홍,1-1,유격수 땅볼,무사 1루 1:5,1사 1루 1:5,1.20,-0.380,85.1,87.8,-0.027
205943,2021,79608,안치홍,LT,내,2021-08-25,KIA,5초,멩덴,안치홍,0-0,포수 땅볼,1사 4:6,2사 4:6,0.79,-0.180,78.4,80.3,-0.019
205944,2021,79608,안치홍,LT,내,2021-08-25,KIA,7초,박진태,안치홍,1-1,유격수 내야안타,"1사 1,2루 4:7",1사 만루 4:7,2.30,0.679,84.9,77.0,0.079


In [ ]:
same_name_fail_list

[]

In [ ]:
data.to_csv('/content/drive/MyDrive/crawling_data.csv')

In [ ]:
hitter.loc[hitter['PCODE']==62797]

,GYEAR,PCODE,GAMENUM,PA,AB,BA,HIT,HR,TOTB,SLG,SF,BB,KK,IB,HP,GD,NAME,T_ID,POSITION,AGE_VA,MONEY,BIRTH
45,2018,62797,73,236,209,0.254,53,2,68,0.325,1,22,32,2,4,8,양성우,HH,외,29,10000만원,0
316,2019,62797,56,129,119,0.168,20,1,26,0.218,0,9,28,0,0,2,양성우,HH,외,30,8500만원,0
627,2020,62797,5,9,9,0.222,2,0,2,0.222,0,0,3,0,0,0,양성우,HH,외,31,7000만원,0


In [ ]:
hts.loc[hts['GYEAR']==2018,:].loc[hts['G_ID']=='20180522LTSS0',:].loc[hts['PCODE']==79465]

,GYEAR,G_ID,PIT_ID,PCODE,T_ID,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,STADIUM
11066,2018,20180522LTSS0,180522_151937,79465,SS,4,156.84,3.6,2루타,136.82,대구


In [ ]:
hitter.loc[hitter['NAME']=='배영섭']

,GYEAR,PCODE,GAMENUM,PA,AB,BA,HIT,HR,TOTB,SLG,SF,BB,KK,IB,HP,GD,NAME,T_ID,POSITION,AGE_VA,MONEY,BIRTH
269,2018,79465,69,168,147,0.245,36,0,42,0.286,0,16,35,2,3,2,배영섭,SS,외,32,13000만원,0
562,2019,79465,30,80,71,0.225,16,0,17,0.239,1,6,15,0,2,0,배영섭,SK,외,33,12000만원,0


In [ ]:
crawling_data = pd.read_csv('/content/drive/MyDrive/crawling_data.csv')

In [ ]:
cd_result = crawling_data['result'].apply(lambda x : x.split(':')[1].strip() if len(x.split(':'))==2 else x.strip())
cd_result = cd_result.apply(lambda x : re.sub('1루수|2루수|3루수|유격수|포수|우익수|중견수|좌익수|투수', "", x).strip())
crawling_data.loc[:,['result']] = cd_result
crawling_data

In [ ]:
set(cd_result)

{'2루타',
 '3루타',
 '[실책포함]',
 '고의4구',
 '그라운드 홈런',
 '내야안타',
 '땅볼',
 '땅볼 실책',
 '뜬공',
 '몸에 맞는 볼',
 '번트 땅볼',
 '번트 실책',
 '번트 아웃',
 '번트 야수선택',
 '번트 파울 뜬공',
 '번트병살타',
 '번트안타',
 '병살타',
 '병살타 실책',
 '볼넷',
 '삼중살',
 '삼진',
 '삼진 아웃',
 '스트라이크낫아웃 출루',
 '쓰리번트 아웃',
 '안타',
 '야수선택',
 '야수선택(희번)',
 '인필드 뜬공',
 '직선타 실책',
 '직선타 아웃',
 '타격방해',
 '타구맞음',
 '파울 뜬공',
 '플라이 실책',
 '홈런',
 '희생번트',
 '희생번트 실책',
 '희생플라이',
 '희생플라이 실책'}

In [ ]:
result_list = ['2루타','3루타','그라운드 홈런','내야안타','땅볼','뜬공','번트 땅볼','번트 아웃','번트 야수선택','번드 파울 뜬공','번트병살타','번트안타','병살타','삼중살','쓰리번트 아웃','안타','야수선택','야수선택(희번)','인필드 뜬공','직선타 아웃','파울 뜬공','홈런','희생번트','희생플라이']


result_list

['2루타',
 '3루타',
 '그라운드 홈런',
 '내야안타',
 '땅볼',
 '뜬공',
 '번트 땅볼',
 '번트 아웃',
 '번트 야수선택',
 '번드 파울 뜬공',
 '번트병살타',
 '번트안타',
 '병살타',
 '삼중살',
 '쓰리번트 아웃',
 '안타',
 '야수선택',
 '야수선택(희번)',
 '인필드 뜬공',
 '직선타 아웃',
 '파울 뜬공',
 '홈런',
 '희생번트',
 '희생플라이']

In [ ]:
set(hts['HIT_RESULT'])

{'1루타',
 '2루타',
 '3루타',
 '내야안타(1루타)',
 '땅볼아웃',
 '번트아웃',
 '번트안타',
 '병살타',
 '삼중살타',
 '야수선택',
 '인필드플라이',
 '직선타',
 '파울플라이',
 '플라이',
 '홈런',
 '희생번트',
 '희생플라이'}

In [ ]:
hit_1 = ['']
hit_2 = ['2루타']
hit_3 = ['3루타']
homerun = ['홈런', '그라운드 홈런']
hit_infield = []
hit_bunt
out_ground
out_bunt
out_double
out_triple
out_fly_infield
out_fly_linedrive
out_fly_foul
out_fly
sacrifice_bunt
sacrifice_fly

In [ ]:
crawling_data.loc[crawling_data['result'].apply(lambda x : x in result_list),:]

,Unnamed: 0,GYEAR,PCODE,NAME,T_ID,POSITION,day,vs,inn,pitcher,batter,P,result,before,after,LEV,REa,WPs,WPe,WPa
0,0,2018,60100,백창수,HH,외,2018-03-27,NC,8초,이민호,백창수,1-0,땅볼,무사 6:9,1사 6:9,0.88,-0.258,93.1,95.3,-0.022
2,2,2018,60100,백창수,HH,외,2018-03-28,NC,4초,구창모,백창수,0-0,땅볼,2사 1:2,이닝종료 1:2,0.51,-0.117,67.4,68.8,-0.013
4,4,2018,60100,백창수,HH,외,2018-03-30,SK,2말,김태훈,백창수,0-1,직선타 아웃,"1사 2,3루 1:1","2사 2,3루 1:1",1.57,-0.833,64.1,56.1,-0.080
8,8,2018,60100,백창수,HH,외,2018-03-31,SK,2말,김광현,백창수,2-2,땅볼,1사 3:0,2사 3:0,0.60,-0.180,22.6,21.1,-0.015
9,9,2018,60100,백창수,HH,외,2018-03-31,SK,5말,김광현,백창수,2-1,땅볼,무사 7:0,1사 7:0,0.23,-0.258,2.5,1.9,-0.006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205941,205941,2021,79608,안치홍,LT,내,2021-08-25,KIA,2초,멩덴,안치홍,1-1,땅볼,1사 0:0,2사 0:0,0.65,-0.180,52.4,54.0,-0.016
205942,205942,2021,79608,안치홍,LT,내,2021-08-25,KIA,4초,멩덴,안치홍,1-1,땅볼,무사 1루 1:5,1사 1루 1:5,1.20,-0.380,85.1,87.8,-0.027
205943,205943,2021,79608,안치홍,LT,내,2021-08-25,KIA,5초,멩덴,안치홍,0-0,땅볼,1사 4:6,2사 4:6,0.79,-0.180,78.4,80.3,-0.019
205944,205944,2021,79608,안치홍,LT,내,2021-08-25,KIA,7초,박진태,안치홍,1-1,내야안타,"1사 1,2루 4:7",1사 만루 4:7,2.30,0.679,84.9,77.0,0.079


In [ ]:
crawling_data

,Unnamed: 0,GYEAR,PCODE,NAME,T_ID,POSITION,day,vs,inn,pitcher,batter,P,result,before,after,LEV,REa,WPs,WPe,WPa
0,0,2018,60100,백창수,HH,외,2018-03-27,NC,8초,이민호,백창수,1-0,땅볼,무사 6:9,1사 6:9,0.88,-0.258,93.100,95.3,-0.022
1,1,2018,60100,백창수,HH,외,2018-03-28,NC,2초,구창모,백창수,2-3,볼넷,2사 2루 0:2,"2사 1,2루 0:2",1.10,0.122,71.900,70.7,0.012
2,2,2018,60100,백창수,HH,외,2018-03-28,NC,4초,구창모,백창수,0-0,땅볼,2사 1:2,이닝종료 1:2,0.51,-0.117,67.400,68.8,-0.013
3,3,2018,60100,백창수,HH,외,2018-03-28,NC,7초,유원상,백창수,2-3,땅볼 실책,[실책포함],무사 2:2,무사 1루 2:2,1.540,0.398,50.0,44.100
4,4,2018,60100,백창수,HH,외,2018-03-30,SK,2말,김태훈,백창수,0-1,직선타 아웃,"1사 2,3루 1:1","2사 2,3루 1:1",1.57,-0.833,64.100,56.1,-0.080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205941,205941,2021,79608,안치홍,LT,내,2021-08-25,KIA,2초,멩덴,안치홍,1-1,땅볼,1사 0:0,2사 0:0,0.65,-0.180,52.400,54.0,-0.016
205942,205942,2021,79608,안치홍,LT,내,2021-08-25,KIA,4초,멩덴,안치홍,1-1,땅볼,무사 1루 1:5,1사 1루 1:5,1.20,-0.380,85.100,87.8,-0.027
205943,205943,2021,79608,안치홍,LT,내,2021-08-25,KIA,5초,멩덴,안치홍,0-0,땅볼,1사 4:6,2사 4:6,0.79,-0.180,78.400,80.3,-0.019
205944,205944,2021,79608,안치홍,LT,내,2021-08-25,KIA,7초,박진태,안치홍,1-1,내야안타,"1사 1,2루 4:7",1사 만루 4:7,2.30,0.679,84.900,77.0,0.079


In [ ]:
crawling_data.loc[:,['in']]=
# 뒤에 초말 없애고 숫자로만 표현

,Unnamed: 0,GYEAR,PCODE,NAME,T_ID,POSITION,day,vs,inn,pitcher,batter,P,result,before,after,LEV,REa,WPs,WPe,WPa
0,0,2018,60100,백창수,HH,외,2018-03-27,NC,8초,이민호,백창수,1-0,유격수 땅볼,무사 6:9,1사 6:9,0.88,-0.258,93.100,95.3,-0.022
1,1,2018,60100,백창수,HH,외,2018-03-28,NC,2초,구창모,백창수,2-3,볼넷,2사 2루 0:2,"2사 1,2루 0:2",1.10,0.122,71.900,70.7,0.012
2,2,2018,60100,백창수,HH,외,2018-03-28,NC,4초,구창모,백창수,0-0,3루수 땅볼,2사 1:2,이닝종료 1:2,0.51,-0.117,67.400,68.8,-0.013
3,3,2018,60100,백창수,HH,외,2018-03-28,NC,7초,유원상,백창수,2-3,유격수 땅볼 실책,[실책포함],무사 2:2,무사 1루 2:2,1.540,0.398,50.0,44.100
4,4,2018,60100,백창수,HH,외,2018-03-30,SK,2말,김태훈,백창수,0-1,백창수/10473 : 2루수 직선타 아웃,"1사 2,3루 1:1","2사 2,3루 1:1",1.57,-0.833,64.100,56.1,-0.080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205941,205941,2021,79608,안치홍,LT,내,2021-08-25,KIA,2초,멩덴,안치홍,1-1,2루수 땅볼,1사 0:0,2사 0:0,0.65,-0.180,52.400,54.0,-0.016
205942,205942,2021,79608,안치홍,LT,내,2021-08-25,KIA,4초,멩덴,안치홍,1-1,유격수 땅볼,무사 1루 1:5,1사 1루 1:5,1.20,-0.380,85.100,87.8,-0.027
205943,205943,2021,79608,안치홍,LT,내,2021-08-25,KIA,5초,멩덴,안치홍,0-0,포수 땅볼,1사 4:6,2사 4:6,0.79,-0.180,78.400,80.3,-0.019
205944,205944,2021,79608,안치홍,LT,내,2021-08-25,KIA,7초,박진태,안치홍,1-1,유격수 내야안타,"1사 1,2루 4:7",1사 만루 4:7,2.30,0.679,84.900,77.0,0.079


In [ ]:
# merge 기준 : PCODE, G_ID(더블헤더 분류 가능), INN -> 같은 이닝에 같은 타자가 한번 더 들어올 수 있음 -> 그럼 결국 RESULT도 이용해야하네 -> 결과도 같으면 우짜지.....:( (방법이 없어) -> 그런 경우가 없길 바랄뿐
# 그런 경우가 생긴다면 -> 순서대로 넣어야함 크롤링 데이터 기준으로 진행 되어야하는데..
# 그런 경우가 생기면 -> 원래 2개의 데이터가 4개가 만들어짐 그중에서 2개를 빼는 작업이 필요 -> 순서 둘다 시간순이라 상관 없을듯(그렇게 믿고 싶음(hts도 같은 이닝에서 시간 순서여라..))
# PIT_ID : 날짜_시간 : 시간이면 매우 좋음:)